In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# Data for five large global cities (one from each continent)
data = {
    "Name": ["New York City", "São Paulo", "Tokyo", "Lagos", "Sydney"],
    "Population": [8419600, 12325232, 13929286, 15000000, 5312163],  # Approximate populations
    "Latitude": [40.7128, -23.5505, 35.6895, 6.5244, -33.8688],
    "Longitude": [-74.0060, -46.6333, 139.6917, 3.3792, 151.2093]
}


In [3]:
cities_df = pd.DataFrame(data)

In [4]:
gdf = gpd.GeoDataFrame(
    cities_df, 
    geometry=gpd.points_from_xy(
        cities_df['Longitude'],
          cities_df['Latitude']
          )
        )

In [ ]:
pip install matplotlib -q

In [ ]:
gdf.plot()

In [7]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, color='red', markersize=50)
for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf["Population"]):
    ax.text(x, y, label, fontsize=8, color='black', ha='right')
plt.title("Point Dataset with Labels")
plt.show()

In [21]:
gdf = gdf.set_crs(epsg=4326)

In [ ]:
gdf.crs

In [ ]:
gdf.explore()

In [ ]:
pip install folium mapclassify -q

In [ ]:
gdf.explore()

In [ ]:
pip install leafmap -q

In [ ]:
pip install -U ipywidgets -q

In [30]:
import leafmap

In [31]:
m = leafmap.Map(center=[0, 0], zoom=2)

In [ ]:
m

In [33]:
m.add_gdf(gdf, layer_name="Points")

In [ ]:
m.add_basemap("Esri.WorldImagery")  # Satellite imagery
m.add_basemap("OpenStreetMap")      # OpenStreetMap layer

In [ ]:
gdf.head()

In [ ]:
m = leafmap.Map(center=[0, 0], zoom=2)
m.add_gdf(gdf, layer_name="Points", popup=['Name', 'Population'])
m

In [ ]:
pip install lonboard palettable sidecar -q

In [37]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import shapely
from palettable.colorbrewer.diverging import BrBG_10
from sidecar import Sidecar

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap

In [38]:
url = "https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2019/quarter=1/2019-01-01_performance_mobile_tiles.parquet"

In [39]:
local_path = Path("internet-speeds.parquet")
if local_path.exists():
    gdf = gpd.read_parquet(local_path)
else:
    columns = ["avg_d_kbps", "tile"]
    df = pd.read_parquet(url, columns=columns)

    tile_geometries = shapely.from_wkt(df["tile"])
    tile_centroids = shapely.centroid(tile_geometries)
    gdf = gpd.GeoDataFrame(df[["avg_d_kbps"]], geometry=tile_centroids, crs="EPSG:4326")
    gdf.to_parquet(local_path)

In [ ]:
gdf.head()

In [ ]:
sidecar = Sidecar()

In [ ]:
layer = ScatterplotLayer.from_geopandas(gdf)
m = Map(layer, _height=800)
# with sidecar:
#     display(m)
m

In [42]:
layer.get_fill_color = [0, 0, 200, 200]

In [43]:
min_bound = 5000
max_bound = 50000
download_speed = gdf["avg_d_kbps"]
normalized_download_speed = (download_speed - min_bound) / (max_bound - min_bound)

In [ ]:
normalized_download_speed

In [ ]:
BrBG_10.mpl_colormap


In [46]:
layer.get_fill_color = apply_continuous_cmap(
    normalized_download_speed, BrBG_10, alpha=0.7
)

In [47]:
layer.get_radius = normalized_download_speed * 200
layer.radius_units = "meters"
layer.radius_min_pixels = 0.5

In [ ]:
m